<a href="https://colab.research.google.com/github/Neuralwood-Net/face-recognizer-9000/blob/main/notebooks/Train_plot_and_save_cropped_224.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train networks on 224px color images cropped to face



### Make sure the hardware is in order

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

### Imports

In [2]:
import time
import os
import copy
import sys
import tarfile

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

import torchvision
from torchvision import datasets, models, transforms

from google.cloud import storage

# Placeholder to make it run until the real WoodNet is defined
class WoodNet:
    pass

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

### Fetch and extract the data from the storage bucket

In [4]:
# Define paths separate from the heavy operations below
BASE_PATH = "/content"

BLOB_NAME = "faces/balanced_sampled_cropped_224px_color_70_15_15_split.tar.gz"
zipfilename = os.path.join(BASE_PATH, BLOB_NAME)
extract_to_dir = os.path.join(BASE_PATH, *BLOB_NAME.split(os.path.sep)[:-1])

In [21]:
# Fetch the data
from google.cloud import storage

# Make the required directories
os.makedirs(os.path.join(BASE_PATH, "faces"), exist_ok=True)
os.makedirs(os.path.join(BASE_PATH, "checkpoints"), exist_ok=True)
os.makedirs(os.path.join(BASE_PATH, "logs"), exist_ok=True)

with open(zipfilename, "wb") as f:
    storage.Client.create_anonymous_client().download_blob_to_file(f"gs://tdt4173-datasets/{BLOB_NAME}", f)

In [22]:
# Extract the data
f = tarfile.open(zipfilename, "r:gz")
f.extractall("/content/faces")
f.close()

### Load the data into wrapper classes and apply normalization

In [5]:
class BGR2RGB:
    def __call__(self, im):

        b, g, r = im.split()

        return Image.merge("RGB", (r, g, b))

In [6]:
BATCH_SIZE = 64

data_transforms = transforms.Compose([
    BGR2RGB(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

data_dir = os.path.join(extract_to_dir, "sampled_dataset_balanced_cropped_224")

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms)
                  for x in ['train', 'val', 'test']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes
print(class_names)
print(image_datasets['val'].classes)
print(dataset_sizes)

### Create a helper function to aid in image plotting and show a random sample of the input data

In [7]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

# Get a batch of training data
inputs, classes = next(iter(dataloaders['val']))

inputs, classes = inputs[:8], classes[:8]

print(inputs.shape)

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

### Create a function for training and validation
The following function trains the supplied model with the loss criterion and optimizer supplied, for the specified number of epochs. During training it logs the loss and accuracy for both training and validation. Whenever a better model is found on the validation set, the function saves the model parameters to a file for use for inference later.

In [8]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    modelname = f"{type(model).__name__}-{since}"
    print(f"Training model: `{type(model).__name__}`")

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    num_img = {
        "train": 0,
        "val": 0,
    }
    
    datapoints_per_epoch = 100

    imgs_per_datapoint = {
        "train": int(float(dataset_sizes["train"] / datapoints_per_epoch)),
        "val": int(float(dataset_sizes["val"] / datapoints_per_epoch)),
    }

    print("Images per phase:", imgs_per_datapoint["train"], imgs_per_datapoint["val"])

    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs - 1}")
        print("-" * 10)
        
        with open(os.path.join(BASE_PATH, f"logs/{modelname}.csv"), "a") as f:

            # For each epoch we want to both train and evaluate in that order
            for phase in ["train", "val"]:
                if phase == "train":
                    # Makes the network ready for training, i.e. the parameters can be tuned
                    # and possible Dropouts are activated
                    model.train()
                else:
                    # Makes the network ready for inference, i.e. it is not tunable and will
                    # turn off regularization that might interfere with training
                    model.eval()

                running_loss = 0.0
                running_corrects = 0

                plot_loss = 0
                plot_corrects = 0

                plot_points = 0

                # Iterate over training or validation data
                for inputs, labels in tqdm(dataloaders[phase], desc=f"Epoch: {epoch} ({phase})", file=sys.stdout):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # Reset the gradients before calculating new ones
                    optimizer.zero_grad()

                    
                    # Ask PyTorch to generate computation graph only if in training mode
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)
                        
                        # Only perform update steps if we're training
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()


                    # Save values for statistics and logging
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    
                    plot_loss += loss.item() * inputs.size(0)
                    plot_corrects += torch.sum(preds == labels.data)
                    
                    num_img[phase] += BATCH_SIZE
                    
                    if (num_img[phase] % imgs_per_datapoint[phase]) < (BATCH_SIZE + 1):
                        f.write(f"{time.time()},{epoch},{phase},\
                        {num_img[phase]},{plot_loss / float(imgs_per_datapoint[phase])},\
                        {plot_corrects / float(imgs_per_datapoint[phase])}\n")
                        
                        plot_loss = 0
                        plot_corrects = 0
                        plot_points += 1

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
                print(f"Points plotted: {plot_points}")

                # deep copy the model
                if phase == "val" and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    torch.save(
                        {
                            "loss": epoch_loss,
                            "acc": epoch_acc,
                            "epoch": epoch,
                            "parameters": best_model_wts,
                        },
                        os.path.join(BASE_PATH, f"checkpoints/{modelname}.data"),
                    )
        print()

    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    print(f"Best val Acc: {best_acc:4f}")

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Prepare the home-made CNN – WoodNet
Below is two networks. The first is made by the authors, and is made to be trained from scratch on the training data. The other is fully trained on ImageNet (1000 classes) and fine-tuned on the training data.

In [9]:
class WoodNet(nn.Module):
    size_after_conv = 7 * 7 * 64
    def __init__(self):
        super(WoodNet, self).__init__()
        self.features = nn.Sequential(   
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
            
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(),
        )
        self.classify = nn.Sequential(
            nn.Linear(self.size_after_conv, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1024, len(class_names)),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, self.size_after_conv)
        x = self.classify(x)

        return x

woodnet = WoodNet()
print(woodnet)

### Prepare the pretrained CNN – SqueezeNet
Below is the code for loading in the pretrained SqueezeNet. After it is loaded, the last classification layer is replaced with a one with the correct amount of output classes.

In [9]:
squeezenet = models.squeezenet1_1(pretrained=True, progress=True)
num_ftr = squeezenet.classifier[1].in_channels
squeezenet.classifier[1] = nn.Conv2d(num_ftr, len(class_names), 1, 1)
squeezenet = squeezenet.to(device)
squeezenet

### Train the network
Below is code that instantiates the loss function and optimization method and starts the training.
To train every parameter in SqueezeNet, set `train_full_network = True`, and to `False` if only the last layer is to be trained.

In [11]:
# network = squeezenet
network = woodnet
train_full_network = False

if train_full_network or isinstance(network, WoodNet):
    print("Training full network")
    parameters = network.parameters()
else:
    print("Training only last layer of SqueezeNet")
    parameters = network.classifier[1].parameters()

optimizer = torch.optim.SGD(parameters, lr=0.001, momentum=0.9)
loss_function = nn.CrossEntropyLoss()

train_model(network, loss_function, optimizer, num_epochs=25)

### Upload model weights and training logs to storage

In [ ]:
!ls /content/checkpoints -l

In [ ]:
# Upload checkpoints to storage
client = storage.Client.from_service_account_json("/content/drive/My Drive/## Project/TDT4173 Deep Learning Project-91d3b469375c.json")
bucket = client.get_bucket("tdt4173-datasets")

blob = bucket.blob("checkpoints/SqueezeNet-1605361529.9021263_cropped.data")
filename = "/content/checkpoints/SqueezeNet-1605361529.9021263.data"
blob.upload_from_filename(filename)

blob = bucket.blob("checkpoints/WoodNet-1605365270.1111202_cropped.data")
filename = "/content/checkpoints/WoodNet-1605365270.1111202.data"
blob.upload_from_filename(filename)

In [ ]:
# Upload logs to storage
blob = bucket.blob("logs/SqueezeNet-1605290215.097698.csv")
filename = "/content/logs/SqueezeNet-1605290215.097698.csv"
blob.upload_from_filename(filename)

blob = bucket.blob("logs/SqueezeNet-1605290736.1277423.csv")
filename = "/content/logs/SqueezeNet-1605290736.1277423.csv"
blob.upload_from_filename(filename)

blob = bucket.blob("logs/WoodNet-1605294933.5362356.csv")
filename = "/content/logs/WoodNet-1605294933.5362356.csv"
blob.upload_from_filename(filename)

### Visualize the model performance for some images

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(squeezenet)

In [ ]:
import cv2
inputs = [
    cv2.imread("/content/lars_1.png", cv2.IMREAD_COLOR),
    cv2.imread("/content/morgan_1.png", cv2.IMREAD_COLOR),
    cv2.imread("/content/morgan_2.png", cv2.IMREAD_COLOR),
    cv2.imread("/content/morgan_3.png", cv2.IMREAD_COLOR),
    cv2.imread("/content/ingvar_1.png", cv2.IMREAD_COLOR),
    cv2.imread("/content/dwayne_1.png", cv2.IMREAD_COLOR),
    cv2.imread("/content/kjartan_2.png", cv2.IMREAD_COLOR),
    cv2.imread("/content/faces/sampled_dataset_balanced_244/test/Kjartan/kjartan_video_5_9_augmentation_8.jpg", cv2.IMREAD_COLOR),
]

for i, inp in enumerate(inputs):
    inputs[i] = cv2.cvtColor(cv2.resize(inp, (244, 244)), cv2.COLOR_BGR2RGB)

def get_prediction_image(img, true_lab=None, plot=False):
    assert not plot or (plot and true_lab)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])

    inp = cv2.resize(img, (224, 224)) / 255.0
    inp = inp / std - mean
    inp = inp.transpose((2, 0, 1))

    imgt = torch.Tensor(inp).unsqueeze(0).to(device)

    out = squeezenet(imgt)

    probabilities = F.softmax(out, dim=1)

    prob, class_idx = torch.max(probabilities, dim=1)
    pred = class_names[class_idx]

    if plot:
        plt.imshow(img)
        plt.text(5, 17,   f"Actual   : {true_lab}", color="white", fontsize=14)
        plt.text(5, 34,   f"Predicted: {pred}", color="white", fontsize=14)

    return pred, round(prob.item() * 100, 2), probabilities

get_prediction_image(inputs[2], "Morgan", plot=True)

In [ ]:
plt.imshow(np.concatenate((inputs[1], inputs[2]), axis=1))
(pred, prob), actual = get_prediction_image(inputs[1]), "Morgan"
plt.text(5, 17,   f"Actual   : {actual}", color="white", fontsize=14)
plt.text(5, 34,   f"Predicted: {pred}", color="white", fontsize=14)
plt.text(5, 52,   f"[Certainty ({prob}%)]", color="white", fontsize=12)


(pred, prob), actual = get_prediction_image(inputs[2]), "Morgan"
plt.text(249, 17, f"Actual   : {actual}", color="white", fontsize=14)
plt.text(249, 34, f"Predicted: {pred}", color="white", fontsize=14)
plt.text(249, 52,   f"[Certainty ({prob}%)]", color="white", fontsize=12)

plt.savefig("morgan_crop_plot.png")
plt.show();

In [12]:
woodnet.load_state_dict(torch.load("/content/WoodNet-1605365270.1111202_cropped.data")["parameters"])

model = woodnet.to(device).eval()

running_loss = 0
running_corrects = 0

access = set([0, 1, 2])

tp = 0
fp = 0
fn = 0

for inputs, labels in tqdm(dataloaders["test"]):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = loss_function(outputs, labels)

        # Save values for statistics and logging
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

        for pred, lab in zip(preds, labels):
            pred, lab = pred.item(), lab.item()
            if lab in access and pred in access:
                tp += 1
            elif lab in access and pred not in access:
                fn += 1
            elif lab not in access and pred in access:
                fp += 1


print(running_loss / dataset_sizes["test"], running_corrects.double() / dataset_sizes["test"])

# Precision
print(tp / (tp + fp))

# Recall
print(tp / (tp + fn))